# Coding a NN from Scratch

## 1. Import MNIST dataset from PyTorch

In [1]:
import torch
import torchvision
import torchvision.datasets as datasets

In [2]:
transform = torchvision.transforms.ToTensor()
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data/', train=False, download=True, transform=transform)

In [3]:
print(mnist_trainset,'\n\n', mnist_testset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor() 

 Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data/
    Split: Test
    StandardTransform
Transform: ToTensor()


In [4]:
# I just want the whole dataset in a np array, I do not want to use PyTorch for anything else than downloading MNIST

batch_size_train = 60000
batch_size_test = 10000

train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=batch_size_test, shuffle=True)

train_examples = enumerate(train_loader) #returns an iterable object that will spit the samples by batch
test_examples = enumerate(test_loader)

# Batch size is equal to the m examples in both sets (train and test), so we only need to iterate them one time
# Both batch indexes = 0 for the same reason stated before
train_batch_idx, (X_train_tensor, y_train_tensor) = next(train_examples)
test_batch_idx, (X_test_tensor, y_test_tensor) = next(test_examples)


In [5]:
print(f'X_train_tensor.shape: {X_train_tensor.shape}, y_train_tensor.shape: {y_train_tensor.shape}')
print(f'X_test_tensor.shape: {X_test_tensor.shape}, y_test.shape: {y_test_tensor.shape}')

X_train_tensor.shape: torch.Size([60000, 1, 28, 28]), y_train_tensor.shape: torch.Size([60000])
X_test_tensor.shape: torch.Size([10000, 1, 28, 28]), y_test.shape: torch.Size([10000])


In [6]:
import numpy as np

# Let's convert those tensors to np arrays only with rows and columns

def tensor_to_vector(tensor):
    vector_nd = np.array(tensor)
    vector = vector_nd.reshape(len(tensor),-1)
    return vector

X_train = tensor_to_vector(X_train_tensor)
X_test = tensor_to_vector(X_test_tensor)
y_train = np.array(y_train_tensor)
y_test = np.array(y_test_tensor)

In [7]:
print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')
print(f'X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}')

X_train.shape: (60000, 784), y_train.shape: (60000,)
X_test.shape: (10000, 784), y_test.shape: (10000,)


In [11]:
# We want examples by columns so let's transpose the Xs
X_train = X_train.T
X_test = X_test.T

print(f'X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}')
print(f'X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}')

X_train.shape: (784, 60000), y_train.shape: (60000,)
X_test.shape: (784, 10000), y_test.shape: (10000,)


In [ ]:
# Let's initalize all our parameters